When thinking in Foundational Models we should think on a model trained on generic visual patterns that can be transferred to new tasks without task-specific training. In Earth Observation (EO) it means, it doesnt know about land cover, physics, vegetation types. It only know textures, shapes, contrast. IT is powerful but dangerous.

SAM is zero-shot ; object agnostic ; scale-agnostic ; spectrally blind. Therefore, it creates the perfect tension: it segments beautifully, but it understands nothing environmentally. It segment baesd on texture/shape/contrast alone ignoring spectal signatures (NDVI for vegetation) or physical context (reef vs sandbar distinction in turbid water)

Concept: The Segment Anything Model (SAM) has "seen" 11 billion images, but it doesn't know what a "tree" is. It only knows what an "object" is (texture, edge, contrast).

In this practical we use foundation models as analysis tools, not as models to be trained, therefore no GPU is required.

Instead of loading a downloaded image lets swithc to cloud straming keeping the same areas Hulhudhoo Maldives for coastal/reef segmentation; and Hyde Park/London for urban green space. 

LEts use pystac-client + planetary_computer + stackstac (already in the container)

Do these polygons correspond to meaningful geographic objects?

Could you calculate area from these?

Would a policymaker trust these shapes?
SAM Porposes objects, We test whether those objects are environmentally meaningful.

Data

Tile-based RGB imagery (leafmap basemap)

Two contrasting environments:

Hyde Park (urban green space)

Hulhudhoo, Maldives (reef / lagoon system)

No STAC. No spectral bands. This is intentional.

Part A — Spatial framing (before any model)

Task A1

Display the study area on a map.

Draw or define an AOI.

Answer (Markdown, short paragraph):

What environmental processes dominate this area?
(e.g. vegetation structure, water depth, urban materials, shadows)

Why this matters
Students must define the question before running a model.

Part B — What the model sees

Task B1

Download a small RGB image for the AOI.

Visualise it.

Answer:

What objects are visually distinct?

What boundaries are ambiguous?

What information is missing?

(Students should mention no spectral, no physics, no depth.)

In [ ]:
import leafmap
import os
from samgeo import SamGeo
import pystac_client
import planetary_computer
import rioxarray

In [2]:
# Bounding boxes (W, S, E, N) in lon/lat (EPSG:4326)

AOI = {
    "Hulhudhoo_Maldives": (73.20, 3.82, 73.26, 3.88),
    "HydePark_London": (-0.19, 51.50, -0.15, 51.52),
}

AOI


{'Hulhudhoo_Maldives': (73.2, 3.82, 73.26, 3.88),
 'HydePark_London': (-0.19, 51.5, -0.15, 51.52)}

In [5]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

print("Connected to Planetary Computer STAC")


Connected to Planetary Computer STAC


In [6]:
site = "HydePark_London"
bbox = AOI[site]

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2021-04-01/2021-05-15",
    query={"eo:cloud_cover": {"lt": 10}},
)

items = list(search.get_items())
len(items)


2

In [7]:
item = items[0]
item.id, item.datetime, item.properties.get("eo:cloud_cover")


('S2B_MSIL2A_20210423T105619_R094_T30UXC_20210525T005413',
 datetime.datetime(2021, 4, 23, 10, 56, 19, 24000, tzinfo=tzutc()),
 1.047312)

In [11]:
import folium

# Map centered on Hyde Park
m = folium.Map(location=[51.51, -0.17], zoom_start=14, tiles="OpenStreetMap")

# AOI bbox = (W, S, E, N)
west, south, east, north = bbox

# Draw AOI rectangle
folium.Rectangle(
    bounds=[(south, west), (north, east)],
    color="yellow",
    fill=False,
    weight=3,
).add_to(m)

# Draw STAC item footprint (GeoJSON)
folium.GeoJson(
    data=item.geometry,
    name="Sentinel-2 scene footprint",
).add_to(m)

folium.LayerControl().add_to(m)
m


A foundation model can segment anything — but understands nothing about Earth systems.

In [1]:
import os
import leafmap.foliumap as leafmap  # stable inside containers
from samgeo import SamGeo
from samgeo.common import tms_to_geotiff


### PART A

#### DEFINE STUDY AREA

In Earth Observation, where you look matters as much as what you do.

Hyde Park represents an urban green space embedded in infrastructure.
Hulhudhoo represents a coastal reef–lagoon system driven by natural processes.

We will use the same model on both — and observe what changes.

In [2]:
SITE = "HydePark"   # change to "Hulhudhoo" later

AOI_DEFAULT = {
    "HydePark": [-0.19, 51.50, -0.15, 51.52],       # [W, S, E, N]
    "Hulhudhoo": [73.215, 3.835, 73.255, 3.875],    # [W, S, E, N]
}

bbox = AOI_DEFAULT[SITE]
bbox


[-0.19, 51.5, -0.15, 51.52]

Student Questions:

What dominant land-cover types do you expect in this AOI?

What spatial patterns matter here (edges, textures, context)?

Which features are environmental rather than visual?

### PART B

#### Foundational model segmentation (SAM as a blind observer)

SAM is a foundation vision model trained on generic images.

t is:

zero-shot

object-agnostic

scale-agnostic

spectrally blind

It sees shapes, contrast, and texture — not vegetation, water, or land use.

#### Interactive map

In [3]:
import leafmap.foliumap as leafmap

m = leafmap.Map(center=[51.51, -0.17], zoom=16)
m.add_basemap("SATELLITE")
m


#### Download a small GeoTiff (CPU-safe)

In [4]:
import os, urllib.request

ckpt_dir = os.path.expanduser("~/.cache/torch/hub/checkpoints")
os.makedirs(ckpt_dir, exist_ok=True)

ckpt_path = os.path.join(ckpt_dir, "sam_vit_b_01ec64.pth")
url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"

if not os.path.exists(ckpt_path):
    print("Downloading SAM checkpoint...")
    urllib.request.urlretrieve(url, ckpt_path)
    print("Saved to:", ckpt_path)
else:
    print("Checkpoint already exists:", ckpt_path)


Checkpoint already exists: /home/vscode/.cache/torch/hub/checkpoints/sam_vit_b_01ec64.pth


In [5]:
out_dir = "data/week3"
os.makedirs(out_dir, exist_ok=True)

image = os.path.join(out_dir, f"{SITE}_satellite.tif")

# zoom controls resolution & file size (17–18 is usually fine; 19 can get heavy)
tms_to_geotiff(
    output=image,
    bbox=bbox,
    zoom=16,
    source="Satellite",   # uses an online basemap
)

image


The output file data/week3/HydePark_satellite.tif already exists. Use `overwrite=True` to overwrite it.


'data/week3/HydePark_satellite.tif'

What is the approximate spatial resolution implied by the zoom level?

Is this image “true colour”? If not, what does that imply?

(Hint: many basemaps are RGB composites with unknown processing.)

#### PART C

In [6]:
import os
print("GeoTIFF size (MB):", round(os.path.getsize(image)/1e6, 2))


GeoTIFF size (MB): 23.36


#### Run SAM (zero-shot)

SAM is zero-shot segmentation. "Zero-Shot" means "No Training Required."
It produces masks that look correct, but it does not know what they represent environmentally.

We use CPU mode and small images, so it runs locally.

In [9]:
from samgeo import SamGeo


In the past (Supervised Learning), if you wanted to find trees in Hyde Park, you had to:

Download 50 images.

Hand-draw 500 polygons around trees.

Train a model for 3 hours.

Run it.


What we are doing right now IS Zero-Shot:

Download 1 image of Hyde Park.

Load SAM (which has already seen 11 billion images).

Draw one box around the park.

The model instantly segments it perfectly.

Why is this Zero-Shot? Because the model has never been trained on your specific image of Hyde Park. It is transferring its general knowledge to your specific problem with zero additional training

In [7]:
sam = SamGeo(
    model_type="vit_b",
    automatic=False,
    points_per_side=12,       # was 24 → much lighter
    pred_iou_thresh=0.90,
    stability_score_thresh=0.95,
    device="cpu"
)


In [8]:
sam.generate(image)


AttributeError: 'SamGeo' object has no attribute 'mask_generator'

In [ ]:
sam.generate(image)


In [9]:
import psutil, os
print("RAM available (GB):", round(psutil.virtual_memory().available/1e9, 2))
print("RAM total (GB):", round(psutil.virtual_memory().total/1e9, 2))
print("CPU cores:", os.cpu_count())


RAM available (GB): 4.8
RAM total (GB): 8.15
CPU cores: 12


In [1]:
import os
print("GeoTIFF size (MB):", round(os.path.getsize(image)/1e6, 2))


NameError: name 'image' is not defined

In [11]:
import os
import leafmap.foliumap as leafmap
from samgeo import SamGeo
# CORRECT IMPORT: This function lives in the 'common' submodule
from samgeo.common import tms_to_geotiff 

# 1. Define Area (Small AOI for CPU safety)
SITE = "HydePark"
# Use a focused bounding box to avoid memory overflow
bbox = [-0.168, 51.505, -0.165, 51.508] 

# 2. Visualization 
m = leafmap.Map(center=[51.506, -0.166], zoom=17)
m.add_basemap("SATELLITE")
m

In [14]:
import os
import leafmap.foliumap as leafmap
from samgeo import SamGeo
from samgeo.common import tms_to_geotiff 

# 1. Define Area
SITE = "HydePark"
bbox = [-0.168, 51.505, -0.165, 51.508] 

# 2. Visualization
m = leafmap.Map(center=[51.506, -0.166], zoom=17)
m.add_basemap("SATELLITE")

# 3. Download Data 
out_dir = "data/week3"
os.makedirs(out_dir, exist_ok=True)
image = os.path.join(out_dir, f"{SITE}_satellite.tif")
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source="Satellite")

# 4. Initialize SAM for Prompting (The Fix)
sam = SamGeo(
    model_type="vit_b", 
    device="cpu",
    automatic=False  # <--- CRITICAL: Enables 'predictor' mode for box/point prompts
)
import numpy as np  # Essential for geospatial matrix math

# ... (Previous steps 1-4 remain the same) ...

# 5. Generate Masks using a Box Prompt (Corrected)
sam.set_image(image)

# FIX: Explicitly convert the simple list to a NumPy array.
# The structure must be [[min_x, min_y, max_x, max_y]]
# We use 'np.array' so the internal library can use .astype(float) without crashing.
box_prompt = np.array([bbox])

print(f"Prompting SAM with box: {box_prompt}")

# Run prediction
sam.predict(boxes=box_prompt, point_crs="EPSG:4326")

# 6. Save and View
output_mask = os.path.join(out_dir, "segmentation_mask.tif")
sam.save_prediction(output_mask)

m.add_raster(output_mask, layer_name="AI Segmentation", opacity=0.6, palette="viridis")
m

The output file data/week3/HydePark_satellite.tif already exists. Use `overwrite=True` to overwrite it.
Prompting SAM with box: [[-0.168 51.505 -0.165 51.508]]


In [15]:
import rasterio
import numpy as np

# 7. Environmental Analysis: Calculate Area
# We open the mask we just created to count the pixels
with rasterio.open(output_mask) as src:
    mask_data = src.read(1)
    # Get the resolution of a single pixel (e.g., 0.6m x 0.6m)
    res_x, res_y = src.res 
    pixel_area_m2 = res_x * res_y

    # Count pixels identified as "Target" (Value 255 or 1 usually)
    # SAM masks are often binary (0 and 255)
    target_pixels = np.count_nonzero(mask_data)
    
    total_area_m2 = target_pixels * pixel_area_m2
    total_area_ha = total_area_m2 / 10000  # Convert to Hectares

print(f"--- Environmental Metrics ---")
print(f"Detected Feature Area: {total_area_m2:,.2f} m²")
print(f"Detected Feature Area: {total_area_ha:,.4f} Hectares")

--- Environmental Metrics ---
Detected Feature Area: 0.00 m²
Detected Feature Area: 0.0000 Hectares


In [16]:
import numpy as np
import rasterio

# Check if the file exists and what's inside
if os.path.exists(output_mask):
    with rasterio.open(output_mask) as src:
        data = src.read(1)
        unique_values = np.unique(data)
        print(f"File contents (Unique Values): {unique_values}")
        
        if len(unique_values) == 1 and unique_values[0] == 0:
            print("FAILURE: The mask is all zeros. The AI didn't 'see' anything in that box.")
        else:
            print("SUCCESS: We have a mask! The issue might be visualization.")
else:
    print("ERROR: No output file found.")

File contents (Unique Values): [0.]
FAILURE: The mask is all zeros. The AI didn't 'see' anything in that box.


In [17]:
# 1. Define a specific point INSIDE your downloaded area
# (Center of the previous bbox)
point_lat = 51.5065
point_lon = -0.1665
point_coords = [[point_lon, point_lat]]  # Note: Longitude first (X), then Latitude (Y)

# 2. Reset the image
sam.set_image(image)

# 3. Predict using a POINT instead of a BOX
# format: point_coords=[[lon, lat]], point_labels=[1] (1 means "include this", 0 means "exclude")
sam.predict(
    point_coords=point_coords, 
    point_labels=[1], 
    point_crs="EPSG:4326"
)

# 4. Save and Check
output_mask_point = os.path.join(out_dir, "segmentation_point.tif")
sam.save_prediction(output_mask_point)

print("Tried Point Prompt. Checking result...")
# Quick check
with rasterio.open(output_mask_point) as src:
    print(f"New Mask Values: {np.unique(src.read(1))}")

# 5. Visualize
m.add_raster(output_mask_point, layer_name="Point Prompt Result", palette="jet")
m

Tried Point Prompt. Checking result...
New Mask Values: [  0. 255.]


In [18]:
import rasterio
import numpy as np

# 6. Environmental Metrics: From Pixels to Hectares
# Open the Point-Prompt mask you just created
mask_path = "data/week3/segmentation_point.tif"

with rasterio.open(mask_path) as src:
    mask_data = src.read(1)
    
    # Extract resolution from the GeoTIFF metadata
    # transform[0] is pixel width, transform[4] is pixel height (usually negative)
    res_x = abs(src.transform[0])
    res_y = abs(src.transform[4])
    pixel_area_m2 = res_x * res_y
    
    # Count only the 'Target' pixels (Value 255)
    target_pixels = np.count_nonzero(mask_data == 255)
    
    total_area_m2 = target_pixels * pixel_area_m2
    total_area_ha = total_area_m2 / 10000 

print(f"--- RESULTS FOR {SITE} ---")
print(f"Spatial Resolution: {res_x:.2f}m x {res_y:.2f}m per pixel")
print(f"Identified Feature Area: {total_area_ha:.4f} Hectares")

--- RESULTS FOR HydePark ---
Spatial Resolution: 1.19m x 1.19m per pixel
Identified Feature Area: 0.8276 Hectares


In [20]:
import os

# 1. Define paths
out_dir = "data/week3"
mask_path = os.path.join(out_dir, "segmentation_point.tif")
vector_path = os.path.join(out_dir, "segmentation_vectors.geojson")

# 2. Convert Pixel Mask to Vector Polygon
# We use the 'sam' object you initialized earlier
sam.tiff_to_vector(mask_path, vector_path)

# 3. Visualize on the Map
# If 'm' is closed, we recreate it centered on your point
if 'm' not in locals():
    import leafmap.foliumap as leafmap
    m = leafmap.Map(center=[51.5065, -0.1665], zoom=17)
    m.add_basemap("SATELLITE")

# Add the red outline
m.add_geojson(
    vector_path, 
    layer_name="AI Tree Detection", 
    style={"color": "#ff0000", "fillOpacity": 0.0, "weight": 3}  # Red outline, no fill
)
m

In [21]:
import geopandas as gpd

# Load the vector file we just made
gdf = gpd.read_file(vector_path)

print("--- DIAGNOSTIC CHECK ---")
print(f"Number of Polygons found: {len(gdf)}")
print(f"Coordinate Reference System: {gdf.crs}")
print("Geometry preview:")
print(gdf.geometry.head())

# Check if the bounds make sense (should be near London)
print(f"Bounds: {gdf.total_bounds}") 
# London is approx: [-0.2, 51.5, -0.1, 51.6]

--- DIAGNOSTIC CHECK ---
Number of Polygons found: 1
Coordinate Reference System: EPSG:3857
Geometry preview:
0    POLYGON ((-18532.545 6711420.651, -18532.545 6...
Name: geometry, dtype: geometry
Bounds: [ -18557.62798772 6711300.01605825  -18462.07477673 6711420.65055172]


In [22]:
import leafmap.foliumap as leafmap

# 1. Create a BRAND NEW map (m2) to bypass any display cache
m2 = leafmap.Map(center=[51.5065, -0.1665], zoom=17)
m2.add_basemap("SATELLITE")

# 2. Add the vector file again, but with a filled style so it pops
# We use 'add_vector' which is more robust than 'add_geojson'
m2.add_vector(
    vector_path, 
    layer_name="Detected Trees", 
    style={
        "color": "red",       # The Outline
        "weight": 3, 
        "fillColor": "yellow", # The Inside
        "fillOpacity": 0.5
    }
)

# 3. Display the NEW map
m2

In [23]:
image

'data/week3/HydePark_satellite.tif'

In [24]:
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source="Satellite", overwrite=True)

Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to data/week3/HydePark_satellite.tif


In [26]:
m = leafmap.Map(center=[51.51, -0.17], zoom=16)
m.add_basemap("SATELLITE")

m.add_raster(
    image,
    layer_name="Satellite image",
    show=False
)

m.add_layer_control()
m


In [27]:
from samgeo.fast_sam import SamGeo

sam = SamGeo(model="FastSAM-x.pt")

There was an error importing fastsam. To use FAST-SAM, install it as:
	pip install segment-geospatial[fast]


NameError: name 'FastSAM' is not defined